In [1]:
!pip3 install tensorflow-estimator==2.6.0
# !pip install numpy==1.19.2 six==1.15.0 typing-extensions==3.7.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.9/462.9 kB 11.0 MB/s eta 0:00:0000:01
DEPRECATION: mecab-python 0.996-ko-0.9.2 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of mecab-python or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.15.0
    Uninstalling tensorflow-estimator-2.15.0:
      Successfully uninstalled tensorflow-estimator-2.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0.post1 requires numpy<2.0.0,>=1.23.5, but you have numpy 1.21.4 which is incompatible.
tensorflow 2.15.0.post1 requires tensorflow-estimator<2.16,>=2.15.0,

In [2]:
# Tensorflow와 Tensoflow Hub에 있는 이미 학습된 모델을 활용하기 위한 모듈 
import tensorflow as tf
import tensorflow_hub as hub

# 이미지를 다운로드하는 데에 사용하는 모듈
import matplotlib.pyplot as plt
import tempfile 
from six.moves.urllib.request import urlopen
from six import BytesIO

# 이미지에 선과 폰트를 넣기 위한 모듈
import numpy as np
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps

# Detection하는 데에 소요된 시간을 점검하기 위한 모듈
import time

/opt/conda/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


ImportError: cannot import name 'estimator_export' from 'tensorflow.python.util.tf_export' (/opt/conda/lib/python3.9/site-packages/tensorflow/python/util/tf_export.py)

In [3]:
def display_image(image):
  fig = plt.figure(figsize=(20, 15))
  plt.grid(False)
  plt.imshow(image)

In [4]:
def download_and_resize_image(url, new_width=256, new_height=256, display=False): 
  
  _, filename = tempfile.mkstemp(suffix=".jpg")
  response = urlopen(url)
  image_data = response.read()
  image_data = BytesIO(image_data)  # 이미지 읽기
  pil_image = Image.open(image_data)
  pil_image = ImageOps.fit(pil_image, (new_width, new_height), Image.ANTIALIAS) # 이미지 조정
  pil_image_rgb = pil_image.convert("RGB") #RBG로 변환
  pil_image_rgb.save(filename, format="JPEG", quality=90) # 이미지 저장
  print("Image downloaded to %s." % filename)
  if display:
    display_image(pil_image)
  return filename


In [ ]:
def draw_bounding_box_on_image(image, 
                               ymin, xmin, ymax, xmax,
                               color,
                               font,
                               thickness=4,
                               display_str_list=()):
  
  draw = ImageDraw.Draw(image)
  im_width, im_height = image.size
  (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
                                ymin * im_height, ymax * im_height)

# BBox 테두리 그리기
  draw.line([(left, top), (left, bottom), (right, bottom), (right, top),
             (left, top)],
            width=thickness,
            fill=color)

  display_str_heights = [font.getsize(ds)[1] for ds in display_str_list]
  
  total_display_str_height = (1 + 2 * 0.05) * sum(display_str_heights)

  if top > total_display_str_height:
    text_bottom = top
  else:
    text_bottom = top + total_display_str_height
  
  for display_str in display_str_list[::-1]:
    text_width, text_height = font.getsize(display_str)
    margin = np.ceil(0.05 * text_height)
    draw.rectangle([(left, text_bottom - text_height - 2 * margin),
                    (left + text_width, text_bottom)],
                   fill=color)
    draw.text((left + margin, text_bottom - text_height - margin),
              display_str,
              fill="black",
              font=font)
    text_bottom -= text_height - 2 * margin

In [ ]:
# BBox 그리기
def draw_boxes(image, boxes, class_names, scores, max_boxes=10, min_score=0.1): 
  colors = list(ImageColor.colormap.values())

  try:
    font = ImageFont.truetype("/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Regular.ttf",
                              25)
  except IOError:
    print("Font not found, using default font.")
    font = ImageFont.load_default()

# BBox 그리기 적용
  for i in range(min(boxes.shape[0], max_boxes)):
    if scores[i] >= min_score:
      ymin, xmin, ymax, xmax = tuple(boxes[i])  # 박스 좌표값
      display_str = "{}: {}%".format(class_names[i].decode("ascii"),
                                     int(100 * scores[i]))
      color = colors[hash(class_names[i]) % len(colors)]
      image_pil = Image.fromarray(np.uint8(image)).convert("RGB")
      draw_bounding_box_on_image(
          image_pil,
          ymin,
          xmin,
          ymax,
          xmax,
          color,
          font,
          display_str_list=[display_str])
      np.copyto(image, np.array(image_pil))
  return image

In [ ]:
image_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/d/db/Torneo_beach_handball.jpg/640px-Torneo_beach_handball.jpg"
downloaded_image_path = download_and_resize_image(image_url, 1280, 856, True)  # new_width와 new_height를 변경해 보세요.

In [ ]:
module_handle = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"  

detector = hub.load(module_handle).signatures['default'] # detector에 사용할 모듈 저장

In [ ]:
def load_img(path):
  img = tf.io.read_file(path)
  img = tf.image.decode_jpeg(img, channels=3)
  return img

In [ ]:
def run_detector(detector, path):
  img = load_img(path) 

  converted_img  = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]

# 소요시간 측정
  start_time = time.time()
  result = detector(converted_img)
  end_time = time.time()

  result = {key:value.numpy() for key,value in result.items()}

# 이미지 추론 개수와 추론 시간 출력
  print("Found %d objects." % len(result["detection_scores"]))
  print("Inference time: ", end_time-start_time)

# 이미지 내 박스로 entity, scores를 추가하여 출력
  image_with_boxes = draw_boxes(
      img.numpy(), result["detection_boxes"],
      result["detection_class_entities"], result["detection_scores"])

  display_image(image_with_boxes)

In [ ]:
run_detector(detector, downloaded_image_path)

In [ ]:
# # 이미지 불러오기
# #image_url = "https://www.google.co.kr/imgres?imgurl=https%3A%2F%2Fimg.theqoo.net%2Fimg%2FHYFQZ.png&tbnid=HP5t0T4gtLb6uM&vet=12ahUKEwip-ti8r7KDAxXeVPUHHTGPD7MQMyglegUIARCwAQ..i&imgrefurl=https%3A%2F%2Ftheqoo.net%2Fsquare%2F2474393590&docid=Te7Eu0VWgOhN6M&w=1163&h=486&q=%ED%83%91%EA%B1%B4%EB%B9%84%EC%B9%98%EB%B0%9C%EB%A6%AC%EB%B3%BC&hl=ko&ved=2ahUKEwip-ti8r7KDAxXeVPUHHTGPD7MQMyglegUIARCwAQ"
# #downloaded_image_path = download_and_resize_image(image_url, 1280, 856, True)  # new_width와 new_height를 변경해 보세요.
# import requests
# from PIL import Image
# from io import BytesIO

# # 이미지 URL
# url = "https://www.artinsight.co.kr/data/tmp/2209/20220923212833_maxwbbqk.jpg"

# # 이미지를 불러옵니다.
# response = requests.get(url)

# # BytesIO 객체를 만들고 불러온 이미지를 열어 PIL Image 객체로 변환합니다.
# img = Image.open(BytesIO(response.content))

# # 이미지를 JPG 형식으로 저장합니다.
# top_gum_img=img.save("local_image.jpg")
image_url = "https://www.artinsight.co.kr/data/tmp/2209/20220923212833_maxwbbqk.jpg"
downloaded_image_path = download_and_resize_image(image_url, 1280, 856, True)  # new_width와 new_height를 변경해 보세요.




In [ ]:
# 테스트
run_detector(detector, downloaded_image_path)

In [ ]:
# 문제 2-2. 모델 불러오기
# !pip install -U pycocotools
# !pip install keras-cv==0.6.4
# !pip install keras-nlp==0.6.3
!pip install --upgrade pip
!pip install --upgrade tensorflow
import tensorflow as tf
print(tf.__version__)

import keras_cv
import keras
import numpy as np
from keras_cv import visualization

In [ ]:
# 문제 2-2. 테스트 
# !git clone https://github.com/qqwweee/keras-yolo3.git
# %cd keras-yolo3
# !wget https://pjreddie.com/media/files/yolov3.weights
# !python convert.py yolov3.cfg yolov3.weights model_data/yolo.h5

from keras.layers import BatchNormalization
from keras_yolo3.yolo import YOLO
from PIL import Image
!cd keras-yolo3

# YOLOv3 모델을 불러옵니다.
yolo = YOLO(model_path='model_data/yolo.h5', 
             anchors_path='model_data/yolo_anchors.txt', 
             classes_path='model_data/coco_classes.txt')

# 탐지를 수행할 이미지를 불러옵니다.
image_path = 'path_to_your_image.jpg'  # 이 부분을 탐지하고자 하는 이미지의 경로로 변경해주세요.
image = Image.open(image_path)

# 이미지에 대해 탐지를 수행합니다.
result_image = yolo.detect_image(image)

# 결과 이미지를 출력합니다.
result_image.show()


In [ ]:
# 문제 2-2. 결과 비교
print("1번째 모델인 Inception ResNet을 사용한 Faster RCNN은 조금 느리지만 높은 정확성을 보였습니다.") 
print("2번째 모델인 ...")

## yolo모델을 불러오려했는대 호환도완되고 오류가 뜨고해서 버전이 꼬임과 동시에 전에했던 작업도 안되서 결과없이 첨부하게되었다. 